In [0]:
def mount_adls(storage_account_name, container_name):
    """
    Mounts Azure Data Lake Storage at the container level.
    
    Args:
        storage_account_name (str): Name of the storage account
        container_name (str): Name of the container
    """
    # Get secrets from key vault
    client_id = dbutils.secrets.get(scope="udemy-storageaccount-databricks-scope", key="ClientID")
    tenant_id = dbutils.secrets.get(scope="udemy-storageaccount-databricks-scope", key="TenantID")
    client_secret = dbutils.secrets.get(scope="udemy-storageaccount-databricks-scope", key="ClientSecret")

    # Configure Azure Storage access using OAuth
    configs = {
        "fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id": client_id,
        "fs.azure.account.oauth2.client.secret": client_secret,
        "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    }
    
    # Construct the mount point and source
    mount_point = f'/mnt/{container_name}'
    source = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/'

    # Unmount if already mounted
    try:
        dbutils.fs.unmount(mount_point)
        print(f"Unmounted existing mount point: {mount_point}")
    except Exception as e:
        pass  # Mount point doesn't exist

    # Mount storage account
    try:
        dbutils.fs.mount(
            source=source,
            mount_point=mount_point,
            extra_configs=configs
        )
        print(f"Successfully mounted at: {mount_point}")
        
        # Verify mount by listing contents
        contents = dbutils.fs.ls(mount_point)
        print("\nAvailable directories:")
        for item in contents:
            print(f"- {item.name}")
            
    except Exception as e:
        print(f"Error mounting storage: {str(e)}")

# Usage:
mount_adls('synapsecoursedatalakes', 'udemy')

/mnt/udemy has been unmounted.
Unmounted existing mount point: /mnt/udemy
Successfully mounted at: /mnt/udemy

Available directories:
- Presentation/
- Processed/
- Raw/


In [0]:
# List files
display(dbutils.fs.ls('/mnt/udemy/'))

path,name,size,modificationTime
dbfs:/mnt/udemy/Presentation/,Presentation/,0,1735592115000
dbfs:/mnt/udemy/Processed/,Processed/,0,1735592105000
dbfs:/mnt/udemy/Raw/,Raw/,0,1735592100000


In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/mnt/udemy/,abfss://udemy@synapsecoursedatalakes.dfs.core.windows.net,
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/mnt/synapsecoursedatalakes/udemy/raw,abfss://udemy@synapsecoursedatalakes.dfs.core.windows.net/raw,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/synapsecoursedatalakes/udemy/processed,abfss://udemy@synapsecoursedatalakes.dfs.core.windows.net/processed,
/Volume,DbfsReserved,
/mnt/synapsecoursedatalakes/udemy/presentation,abfss://udemy@synapsecoursedatalakes.dfs.core.windows.net/presentation,
